In [1]:
import instaloader
import pandas as pd
import time
from itertools import islice

In [3]:
# user agent
# handle login
# proxy rotation

In [27]:
import json
import random

def get_random_proxy():
    with open('proxies.json', 'r') as f:
        proxies = json.load(f)
    
#         chosen_proxy = random.choice(proxies)
#         ip = chosen_proxy['ip']
#         port = chosen_proxy['port']
#         protocol = chosen_proxy['protocols'][0]  # Assuming you want the first protocol in the list

#         return f"{protocol}://{ip}:{port}"
    proxy = random.choice(proxies)
    port = proxy['port']
    ip = proxy['ip']

    # print(f"Setting a random proxy: {proxy}")
    # print(f"Your proxy PORT is {port}")
    print(f"Your proxy is IP:{ip} and your PORT:{port}")
    
    return proxy, ip, port

# get_random_proxy()

def get_current_ip(driver):
    # Navigate to httpbin's IP check service
    driver.get("http://www.httpbin.org/ip")
    
    # Extract the IP address from the page content
    ip_info = driver.find_element_by_tag_name("pre").text
    ip_address = ip_info.split(":")[1].strip().replace('"', '').replace('}', '').strip()
    
    print(f"Your current IP address is {ip_address}")
    
    return ip_address

In [31]:
def set_opts():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")  # Optional: Use incognito mode which doesn't use cache
    chrome_options.add_argument("--disable-cache")  # Disable the cache
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-application-cache")
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument(gen_ua(browsers, os, min_per))
    
    proxy_, ip, port = get_random_proxy()
    
    proxy_settings = {
        "proxyType": "manual",
        "sslProxy": f"{ip}:{port}"
    }

    caps = chrome_options.to_capabilities()
    print(caps)
    caps['proxy'] = proxy_settings
    print(caps['proxy'])
    
    return chrome_options, caps

In [35]:
import random
from fake_useragent import UserAgent


browsers = ["random", "chrome", "edge", "google", "google chrome", "firefox", "ff", "safari"]

os = ["windows", "macos", "linux"]

min_per = [0.5, 0.6, 0.7, 0.8, 0.9, 1.1, 1.2, 1.3, 1.5]

def gen_ua(browsers, os, min_per):
#     b = random.choice(browsers)
    os = random.choice(os)
    min_per = random.choice(min_per)
    ua = UserAgent(browsers=browsers, os=os, min_percentage=min_per)
    ua = ua.random
    ua = 'user-agent='+ua
    return ua


ua_list = []

for i in range(0,1000):
    ua = gen_ua(browsers, os, min_per)
    ua_list.append(ua)
    
print(gen_ua(browsers, os, min_per))

user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0


In [4]:
class InstagramScraper:
    
    def __init__(self):
        self.L = instaloader.Instaloader()

    def get_users_from_hashtags(self, hashtags, limit=30):
        all_users = []

        for hashtag in hashtags:
            posts = self.L.get_hashtag_posts(hashtag)

            for post in islice(posts, limit):
                try:
                    username = post.owner_username
                    profile = instaloader.Profile.from_username(self.L.context, username)
                    
                    if not profile.is_private:  # Check if profile is public
                        user_data = {
                            "username": username,
                            "profile_url": f"https://www.instagram.com/{username}/",
                            "email": next((line for line in profile.biography.splitlines() if "email" in line), None),
                            "mobile_phone": next((line for line in profile.biography.splitlines() if any(i.isdigit() for i in line)), None),
                            "name": profile.full_name,
                            "bio": profile.biography
                        }
                        all_users.append(user_data)
                    
                except instaloader.BadResponseException as e:
                    if '401' in str(e):
                        print("HTTP error 401 encountered. Pausing for 10 minutes...")
                        time.sleep(600)  # Pausing for 10 minutes
                    else:
                        print(f"Failed to fetch metadata for post with shortcode {post.shortcode}. Skipping.")
                except Exception as e:
                    print(f"An error occurred: {e}. Skipping.")
                time.sleep(2)

        return all_users

    def save_to_csv(self, user_data, csv_filename='users.csv'):
        new_data = pd.DataFrame(user_data)
        try:
            existing_data = pd.read_csv(csv_filename)
        except FileNotFoundError:
            existing_data = pd.DataFrame()

        combined_data = pd.concat([new_data, existing_data], ignore_index=True)
        combined_data.drop_duplicates(subset='username', inplace=True)
        combined_data.to_csv(csv_filename, index=False)

In [18]:
instaloader_dir = !which instaloader
instaloader_dir

['/Users/vzucher/.pyenv/versions/lewagon/bin/instaloader']

In [7]:
def run_all(tags):

    scraper = InstagramScraper()
    hashtags = tags  # Replace with your list of hashtags
    user_data = scraper.get_users_from_hashtags(hashtags)
    scraper.save_to_csv(user_data)

In [ ]:
tags = ['tatuagem']
run_all(tags)

In [26]:
import instaloader
import pandas as pd
import time
import json
import random
from itertools import islice

class InstagramScraper:
    
    def __init__(self, username=None, password=None):
        self.L = instaloader.Instaloader()
        if username and password:
            try:
                self.L.load_session_from_file(username)
            except FileNotFoundError:
                self.L.context.log("Session file does not exist yet - Logging in.")
            
            if not self.L.context.is_logged_in:
                self.L.context.log("Logging in as {}...".format(username))
                self.L.login(username, password)
                self.L.save_session_to_file()
        self.set_rotated_proxy_and_user_agent()

    def get_random_proxy(self):
        with open('proxies.json', 'r') as f:
            proxies = json.load(f)
        
#         chosen_proxy = random.choice(proxies)
#         ip = chosen_proxy['ip']
#         port = chosen_proxy['port']
#         protocol = chosen_proxy['protocols'][0]  # Assuming you want the first protocol in the list

#         return f"{protocol}://{ip}:{port}"
        return random.choice(proxies)

    def get_random_user_agent(self):
        user_agents = pd.read_csv('user_agent.csv')
#         return random.choice(user_agents['user_agent'])
        return random.choice(user_agents).strip()

    def set_rotated_proxy_and_user_agent(self):
        # Set the proxy
#         proxy = self.get_random_proxy()
#         self.L.context._session.proxies = {
#             "http": proxy,
#             "https": proxy
#         }

#         # Set the user agent
#         user_agent = self.get_random_user_agent()
#         self.L.custom_user_agent = user_agent
        
        proxy = self.get_random_proxy()
        user_agent = self.get_random_user_agent()
        self.L.context.session.proxies = {'http': proxy, 'https': proxy}
        self.L.context.session.headers.update({'User-Agent': user_agent})

    def get_users_from_hashtags(self, hashtags, limit=30, max_retries=3):
        all_users = []

        for hashtag in hashtags:
            retries = 0
            while retries < max_retries:
                try:
                    posts = self.L.get_hashtag_posts(hashtag)

                    for post in islice(posts, limit):
                        username = post.owner_username
                        profile = instaloader.Profile.from_username(self.L.context, username)

                        if not profile.is_private:  # Check if profile is public
                            user_data = {
                                "username": username,
                                "profile_url": f"https://www.instagram.com/{username}/",
                                "email": next((line for line in profile.biography.splitlines() if "email" in line), None),
                                "mobile_phone": next((line for line in profile.biography.splitlines() if any(i.isdigit() for i in line)), None),
                                "name": profile.full_name,
                                "bio": profile.biography
                            }
                            all_users.append(user_data)
                    break  # If scraping is successful, break out of the retry loop

                except instaloader.QueryReturnedNotFoundException:
                    retries += 1
                    print("Encountered QueryReturnedNotFoundException. Rotating proxy and user agent and retrying...")
                    time.sleep(15)  # Sleep for 15 seconds
                    self.set_rotated_proxy_and_user_agent()  # Rotate proxy and user agent
                    print("Setting a new proxy and user_agent")

                except instaloader.BadResponseException as e:
                    if 'Redirected to login page' in str(e):
                        print("Redirected to login page. Skipping this user.")
                    else:
                        print(f"Failed to fetch metadata for post with shortcode {post.shortcode}. Skipping.")
                    break  # If this exception is encountered, break out of the retry loop

                except Exception as e:
                    print(f"An error occurred: {e}. Skipping.")
                    break  # If any other exception is encountered, break out of the retry loop

            if retries == max_retries:
                print(f"Reached maximum retries for hashtag {hashtag}. Moving to the next hashtag.")

        return all_users

    def save_to_csv(self, user_data, csv_filename='users.csv'):
        new_data = pd.DataFrame(user_data)
        try:
            existing_data = pd.read_csv(csv_filename)
        except FileNotFoundError:
            existing_data = pd.DataFrame()

        combined_data = pd.concat([new_data, existing_data], ignore_index=True)
        combined_data.drop_duplicates(subset='username', inplace=True)
        combined_data.to_csv(csv_filename, index=False)

# Usage
USERNAME = 'inkerscraper'
PASSWORD = 'inkertattoo'
scraper = InstagramScraper(USERNAME, PASSWORD)
hashtags = ['tatuagem']  # Replace with your list of hashtags
user_data = scraper.get_users_from_hashtags(hashtags)
scraper.save_to_csv(user_data)

Session file does not exist yet - Logging in.
Logging in as inkerscraper...


JSON Query to accounts/login/: Could not find "window._sharedData" in html response. [retrying; skip with ^C]
JSON Query to accounts/login/: Could not find "window._sharedData" in html response. [retrying; skip with ^C]


QueryReturnedNotFoundException: JSON Query to accounts/login/: Could not find "window._sharedData" in html response.